# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from sympy import *

# Provided functions

In [2]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1
    return n, lines, centers

In [3]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')

In [4]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [5]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    num_lines = lines.shape[1]
    vp_candidates = np.zeros((3, 0))
    for i in range(num_lines):
        for j in range(i+1, num_lines):
            line0 = lines[:, i]
            line1 = lines[:, j]
            p = np.cross(line0, line1)
            p = p / p[2]
            vp_candidates = np.append(vp_candidates, p.reshape((3, 1)), axis=1)
    vp_mean = np.mean(vp_candidates, axis = 1)
    print("vp: ", vp_mean)
    return vp_mean

In [6]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>
    vpts_left = vpts[:, 1]
    vpts_right = vpts[:, 0]
    horizon_line = np.cross(vpts_left, vpts_right)
    horizon_line /= np.sqrt(horizon_line[0]**2 + horizon_line[1]**2)
    print(horizon_line)
    return horizon_line

In [7]:
def plot_horizon_line(im, h_line, vpts):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    vpts_left = vpts[:, 1]
    vpts_right = vpts[:, 0]
    bx1 =  vpts_left[0] - 10
    bx2 = vpts_right[0] + 10
    by1 = min(vpts_left[1], vpts_right[1]) - 10
    by2 = max(vpts_left[1], vpts_right[1]) + 10
    
    if h_line[0] < h_line[1]:
        pt1 = np.cross(np.array([1, 0, -bx1]), h_line)
        pt2 = np.cross(np.array([1, 0, -bx2]), h_line)
    else:
        pt1 = np.cross(np.array([0, 1, -by1]), h_line)
        pt2 = np.cross(np.array([0, 1, -by2]), h_line)
    pt1 = pt1 / pt1[2]
    pt2 = pt2 / pt2[2]
    plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'b')
    plt.plot(vpts_left[0], vpts_left[1], 'ro')
    plt.plot(vpts_right[0], vpts_right[1], 'ro')

In [8]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>
    vpt_r = vpts[:, 0].reshape((3, 1))
    vpt_l = vpts[:, 1].reshape((3, 1))
    vpt_inf = vpts[:, 2].reshape((3, 1))
    f, px, py = symbols('f, px, py')
    Kinv = Matrix([[1 / f, 0, -px / f], [0, 1 / f, -py / f], [0, 0, 1]])
    
    # three orthogonal equation
    m1 = vpt_r.T * Kinv.T * Kinv * vpt_l
    m2 = vpt_r.T * Kinv.T * Kinv * vpt_inf
    m3 = vpt_l.T * Kinv.T * Kinv * vpt_inf
    f, px, py = solve([m1[0], m2[0], m3[0]], (f, px, py))[0]
    print(f, px, py)
    return f, px, py

In [9]:
def get_rotation_matrix(f, px, py, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR IMPLEMENTATION>
    vpt_r = vpts[:, 0].reshape((3, 1))
    vpt_l = vpts[:, 1].reshape((3, 1))
    vpt_inf = vpts[:, 2].reshape((3, 1))
    Kinv = np.array([[1 / f, 0, -px / f], [0, 1 / f, -py / f], [0, 0, 1]]).astype(np.float64)
    r1 = Kinv @ vpt_r
    r2 = Kinv @ vpt_inf
    r3 = Kinv @ vpt_l
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)
    R =  np.concatenate((r1, r2, r3), axis=1)
    return R

In [10]:
def estimate_height(im, coords, obj, person_ref, vpts, h_line):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    vpt_inf = vpts[:, 2]
    
    person_ref_top = person_ref[:, 0]
    person_ref_bottom = person_ref[:, 1]

    obj_top = coords[obj][:, 0]
    obj_bottom = coords[obj][:, 1]
    obj_line = np.cross(obj_top, obj_bottom)
    
    bottom_line = np.cross(person_ref_bottom, obj_bottom)
    bottom_line_inter_h_line = np.cross(bottom_line, h_line)
    bottom_line_inter_h_line /= bottom_line_inter_h_line[2]
    
    top_line = np.cross(bottom_line_inter_h_line, person_ref_top)
    top_line_inter_obj_line = np.cross(top_line, obj_line)
    top_line_inter_obj_line /= top_line_inter_obj_line[2]
    
    r = obj_top
    t = top_line_inter_obj_line
    b = obj_bottom
    v = vpt_inf
    
    height_person = 1.6764 # meters
    height_obj = height_person * np.linalg.norm(r-b) * np.linalg.norm(v-t) / np.linalg.norm(t-b) / np.linalg.norm(v-r)
    plt.figure()
    plt.imshow(im)
    plt.plot([person_ref_top[0], person_ref_bottom[0]], [person_ref_top[1], person_ref_bottom[1]])
    plt.plot([obj_top[0], obj_bottom[0]], [obj_top[1], obj_bottom[1]])
    plt.plot([person_ref_bottom[0], obj_bottom[0]], [person_ref_bottom[1], obj_bottom[1]])
    plt.plot([person_ref_top[0], obj_top[0]], [person_ref_top[1], obj_top[1]])
    
    plot_horizon_line(im, h_line, vpts)
    plt.show()
    
    return height_obj
    
    pass

# Main function

In [11]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
#     n, lines, centers = get_input_lines(im)
    # Hardcode three sets of manually chosen lines. Don't want to choose every time since my eyes hurt.
    if(i == 0):
        n = 5
        lines = np.array([[-9.86299837e+00, -4.74885107e+00,  1.46118494e+00, -3.65296236e-01, -1.57077381e+01],
                         [ 5.62556203e+01,  5.55250279e+01,  5.51597316e+01,  1.29314868e+02, 1.11050056e+02],
                         [ 6.74367562e+02, -6.37770146e+03, -1.48524565e+04, -2.91892930e+04, -4.16678233e+03]])
        centers = np.array([[857.61924445, 855.79276327, 852.32244903, 890.13060946, 893.78357182],
                             [138.3742567,  188.05454479, 246.68459067, 228.23713075, 163.94499322],
                             [  1.,           1.,           1.,           1.,           1.        ]])
    elif(i == 1):
        n = 4
        lines = np.array([[ 2.98804815e+00,  7.89698440e+00,  0.00000000e+00, -5.33580027e+00],
                         [ 1.16533878e+02,  1.16533878e+02,  1.16107014e+02,  1.69038152e+02],
                         [-2.47270614e+04, -2.38593507e+04, -2.58276097e+04, -3.73028332e+04]])
        centers = np.array([[398.99458208, 398.56771805, 398.78115006, 525.98662843],
                             [201.95712065, 177.73258744, 222.44659368, 237.28011842],
                             [  1.,           1.,           1.,           1.        ]])
    elif(i == 2):
        n = 3
        lines = np.array([[-1.12011298e+02, -1.69405516e+02, -7.40570564e+01],
                         [-8.33141884e+00,  1.66628377e+01,  2.77713961e+00],
                         [1.09023396e+05, -6.82751557e+02,  2.39758931e+04]])
        centers = np.array([[958.06404989,  10.596585,   331.35621039],
                             [205.1749598,  148.70645432, 202.86067678],
                             [  1.,           1.,           1.,        ]])
    
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plt.figure()
    plt.imshow(im)
    plot_lines_and_vp(im, lines, vpts[:, i])
    plt.show()

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line

plt.figure()
plt.imshow(im)
plot_horizon_line(im, horizon_line, vpts)
plt.show()

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)

# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(f, u, v, vpts)
print(R)
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
# for obj in objects:
#     coords[obj] = get_top_and_bottom_coordinates(im, obj)
#     print(coords[obj])
# person_ref = coords['person']

person_ref = np.array([[628.04729246, 628.12773153],
                         [466.68894009, 512.77874373],
                         [  1.,           1.        ]])
# CSL
coords['CSL building'] = np.array([[603.05303593, 603.8100227 ],
                                     [ 52.89513764, 318.23707312],
                                     [  1.,           1.        ]])

#spike
coords['the spike statue'] = np.array([[603.05303593, 603.8100227 ],
                                         [189.90069192, 473.87040961],
                                         [  1.,           1.        ]])

#lamp post
coords['the lamp posts'] = np.array([[711.36148087, 708.86252483],
                                     [327.8319594,  411.61707502],
                                     [  1.,           1.        ]])


# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, coords, obj, person_ref, vpts, horizon_line)
    print(height)

Getting vanishing point 0
vp:  [1.39703890e+03 2.32946408e+02 1.00000000e+00]
Getting vanishing point 1
vp:  [-181.81389785  219.27762144    1.        ]
Getting vanishing point 2
vp:  [5.39891697e+02 5.61655325e+03 1.00000000e+00]
[-8.65709207e-03  9.99962527e-01 -2.20843384e+02]
-780.673136455971 585.560422942224 341.454588217286
[[-0.71733035  0.00856383  0.69668058]
 [ 0.09591901 -0.98918955  0.11092151]
 [ 0.69009906  0.14639228  0.70875425]]
Estimating height of CSL building
28.436069750348153
Estimating height of the spike statue
11.39060164883104
Estimating height of the lamp posts
4.526484356721848
